In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt

### Lectura de Excels encontrados en el INE o en Datosgob

In [ ]:
poblacion_historica = pd.read_excel('EXCELs Históricos/Poblacion Historica.xlsx')
denuncias_vg = pd.read_excel('EXCELs Históricos/Denuncias VG.xlsx')
asesinatos_vg = pd.read_excel('EXCELs Históricos/Asesinatos VG.xlsx')
vicaria_vg = pd.read_excel('EXCELs Históricos/Violecia Vicaria.xlsx')
allanamientos_periodo = pd.read_excel('EXCELs Históricos/Allanamiento Historico.xlsx')
ciberseguridad = pd.read_excel('EXCELs Históricos/Cibercriminalidad Historico.xlsx')
odio_periodo = pd.read_excel('EXCELs Históricos/Delitos de Odio Historico.xlsx')
delitos_periodo = pd.read_excel('EXCELs Históricos/Delitos Historico.xlsx')
detenciones_periodo = pd.read_excel('EXCELs Históricos/Detenciones Histórico.xlsx')
victimas_sexo = pd.read_excel('EXCELs Históricos/Victimas_Sexo Historico.xlsx')
delitosodio = pd.read_excel('EXCELs Históricos/DelitosOdio.xlsx')

### WebScraping de datos faltantes como % de Pobreza y SMI

In [ ]:
url = "https://datosmacro.expansion.com/demografia/riesgo-pobreza/espana"
resp = requests.get(url)
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "html.parser")

tabla = soup.find("table")

filas = tabla.find_all("tr")

data = []
for fila in filas:
    cols = fila.find_all(["td", "th"])
    cols_text = [col.get_text(strip=True) for col in cols]
    data.append(cols_text)

pobreza_historicos = pd.DataFrame(data)

pobreza_historicos.columns = pobreza_historicos.iloc[0]
pobreza_historicos = pobreza_historicos[1:].reset_index(drop=True)

In [ ]:
url = "https://datosmacro.expansion.com/smi/espana"
resp = requests.get(url)
resp.raise_for_status()

tablas = pd.read_html(resp.text, decimal=",", thousands=".")

salario_historico = tablas[0]

### Limpieza y Normalización de todos los datasets. 

#### - Limpieza y normalización de la Violencia de Género con todos sus datasets

In [ ]:
asesinatos_vg = asesinatos_vg.rename(columns={'Export date and time: 2025-11-25 09:41:30': 'Fecha', 'Unnamed: 1': 'Feminicidios_Totales'})
denuncias_vg = denuncias_vg.rename(columns={'Export date and time: 2025-11-25 09:40:52': 'Fecha', 'Unnamed: 1': 'Denuncias_de_la_Victima', 'Unnamed: 2': 'Denuncias_de_la_Familia', 'Unnamed: 3': 'Atestados_Policiales', 'Unnamed: 4': 'Atestados_Policiales_Familia', 'Unnamed: 5': 'Intervencion_Policial', 'Unnamed: 6': 'Parte_de_lesiones'})
vicaria_vg = vicaria_vg.rename(columns={'Export date and time: 2025-11-25 09:42:11': 'Fecha', 'Unnamed: 1': 'Menores_Victimas'})

In [ ]:
vicaria_vg = vicaria_vg.drop(columns=['Unnamed: 2'])
denuncias_vg = denuncias_vg.drop(columns=['Unnamed: 7'])
asesinatos_vg = asesinatos_vg.drop(columns=['Unnamed: 2'])

denuncias_vg["Fecha"] = denuncias_vg["Fecha"].str.replace("Año ", "", regex=False)

In [ ]:
asesinatos_vicaria = pd.merge(asesinatos_vg, vicaria_vg, on= 'Fecha', how= 'outer')

In [ ]:
asesinatos_vicaria = asesinatos_vicaria.dropna()

In [ ]:
vg = pd.merge(asesinatos_vicaria, denuncias_vg, on= 'Fecha', how= 'outer')

- Eliminación de nulos que no necesitamos

In [ ]:
vg = vg.dropna()

In [ ]:
vg = vg.iloc[0:13]

- Convertir todo a numérico

In [ ]:
for col in vg.columns:
    try:
        vg[col] = vg[col].astype(float)
        
        if (vg[col] % 1 == 0).all():
            vg[col] = vg[col].astype(int)

    except:
        pass


#### - Limpieza y normalización de la Población histórica

In [ ]:
poblacion_historica = poblacion_historica.rename(columns={'Unnamed: 1': 2022, 'Unnamed: 2': 2021, 'Unnamed: 3': 2020, 'Unnamed: 4': 2019, 'Unnamed: 5': 2018, 'Unnamed: 6': 2017, 'Unnamed: 7': 2016, 'Unnamed: 8': 2015, 'Unnamed: 9': 2014  })
poblacion_historica = poblacion_historica.dropna()
poblacion_historica = poblacion_historica.T

In [ ]:
poblacion_historica = poblacion_historica.rename(columns={6: 'Fecha', 8: 'Poblacion_total'})
poblacion_historica['Fecha'] = poblacion_historica['Fecha'].astype(str).str.replace('.0','')
poblacion_historica.reset_index(drop=True, inplace=True)

In [ ]:
poblacion_historica = poblacion_historica.iloc[1:10]

In [ ]:
for col in poblacion_historica.columns:
    try:
        poblacion_historica[col] = poblacion_historica[col].astype(float)
        
        if (poblacion_historica[col] % 1 == 0).all():
            poblacion_historica[col] = poblacion_historica[col].astype(int)

    except:
        pass


- Unión entre Violencia de Género y Población

In [ ]:
historico = pd.merge(poblacion_historica, vg, on='Fecha', how='outer')

In [ ]:
historico.loc[(historico['Fecha'] == 2023) & (historico['Poblacion_total'].isna()),
              'Poblacion_total'] = 48381240

historico.loc[(historico['Fecha'] == 2024) & (historico['Poblacion_total'].isna()),
              'Poblacion_total'] = 48619695

#### - Limpieza y normalizacióin del Umbral de Pobreza

In [ ]:
pobreza_historicos = pobreza_historicos.drop(columns=['Umbral persona','Umbral persona', 'Umbral hogar', 'Umbral hogar'])

In [ ]:
pobreza_historicos = pobreza_historicos.rename({'Personas en riesgo de pobreza': 'Personas_en_riesgo', '% Riesgo Pobreza': ' pct_Riesgo_Pobreza'})

In [ ]:
for col in pobreza_historicos.columns:
    if col != 'Fecha':
        pobreza_historicos[col] = pobreza_historicos[col].astype(str).str.replace(r'[^0-9\.]', '', regex=True)

In [ ]:
for col in pobreza_historicos.columns:
    try:
        pobreza_historicos[col] = pobreza_historicos[col].astype(float)
        
        if (pobreza_historicos[col] % 1 == 0).all():
            pobreza_historicos[col] = pobreza_historicos[col].astype(int)

    except:
        pass

In [ ]:
pobreza_historicos = pobreza_historicos.apply(pd.to_numeric, errors='coerce')


#### - Limpieza y normalización del Salario Minimo

In [ ]:
salario_historico = salario_historico.drop(columns=['SMI Mon. Local', 'SMI'])
salario_historico = salario_historico.rename(columns={'SMI.1': 'SMI'})
salario_historico['Fecha'] = salario_historico['Fecha'].str.extract(r'(\d{4})').astype(int)
salario_historico = salario_historico.drop_duplicates(subset='Fecha', keep='first')

In [ ]:
salario_historico['SMI'] = (
    salario_historico['SMI']
    .astype(str)
    .str.replace('€', '', regex=False)        
    .str.replace('.', '', regex=False)        
    .str.replace(',', '.', regex=False)       
)

In [ ]:
for col in salario_historico.columns:
    try:
        salario_historico[col] = salario_historico[col].astype(float)
        
        if (salario_historico[col] % 1 == 0).all():
            salario_historico[col] = salario_historico[col].astype(int)

    except:
        pass

#### - Limpieza y normalización del Allanamiento Histórico

In [ ]:
allanamientos_periodo = allanamientos_periodo.dropna()
allanamientos_periodo = allanamientos_periodo.rename(columns={'ALLANAMIENTO / USURPACIÓN DE INMUEBLES': 'Fecha', 'Unnamed: 1': ' Allanamientos_totales'})

In [ ]:
allanamientos_periodo = allanamientos_periodo.iloc[1:12]

In [ ]:
for col in allanamientos_periodo.columns:
    try:
        allanamientos_periodo[col] = allanamientos_periodo[col].astype(float)
        
        if (allanamientos_periodo[col] % 1 == 0).all():
            allanamientos_periodo[col] = allanamientos_periodo[col].astype(int)

    except:
        pass

#### - Limpieza y normalización de la Cibercriminalidad

In [ ]:
ciberseguridad = ciberseguridad.dropna()
ciberseguridad = ciberseguridad.rename(columns={'PORTAL ESTADÍSTICO DE CRIMINALIDAD': 'Fecha','Unnamed: 1': 'Acceso_Ilicito', 'Unnamed: 2': 'Amenazas', 'Unnamed: 3': 'Honor', 'Unnamed: 4': 'Propiedad_Industrial', 'Unnamed: 5': 'Delitos_Sexuales', 'Unnamed: 6': 'Falsificacion_Informatica', 'Unnamed: 7': 'Fraude_Informatico', 'Unnamed: 8': 'Interferencia_de_Datos', 'Unnamed: 9': 'Total_cibercriminalidad'})
ciberseguridad = ciberseguridad.iloc[1:12]

In [ ]:
for col in ciberseguridad.columns:
    try:
        ciberseguridad[col] = ciberseguridad[col].astype(float)
        
        if (ciberseguridad[col] % 1 == 0).all():
            ciberseguridad[col] = ciberseguridad[col].astype(int)

    except:
        pass

#### - Limpieza y normalización de Delitos de Odio

In [ ]:
odio_periodo = odio_periodo.dropna()
odio_periodo = odio_periodo.rename(columns={'PORTAL ESTADÍSTICO DE CRIMINALIDAD': 'Fecha','Unnamed: 1': 'Campo_Futbol', 'Unnamed: 2': 'Centro_Religioso', 'Unnamed: 3': 'Espacios_Abiertos', 'Unnamed: 4': 'Establecimientos', 'Unnamed: 5': 'Instalaciones_no_deportivas', 'Unnamed: 6': 'Vias_de_Comunicacion', 'Unnamed: 7': 'Viviendas', 'Unnamed: 8': 'Medios_de_Transporte', 'Unnamed: 9': 'Espacios_Desconocidos', 'Unnamed: 10': 'Total_de_Delitos_de_Odio'})
odio_periodo = odio_periodo.iloc[1:12]

In [ ]:
odio_periodo = odio_periodo.drop(columns=['Espacios_Abiertos'])

In [ ]:
for col in odio_periodo.columns:
    try:
        odio_periodo[col] = odio_periodo[col].astype(float)
        
        if (odio_periodo[col] % 1 == 0).all():
            odio_periodo[col] = odio_periodo[col].astype(int)

    except:
        pass

In [ ]:
delitosodio = delitosodio.dropna()

In [ ]:
delitosodio = delitosodio.rename(columns={'PORTAL ESTADÍSTICO DE CRIMINALIDAD': 'Fecha','Unnamed: 1': 'Antigitanismo', 'Unnamed: 2': 'Antisemitismo', 'Unnamed: 3': 'Aporofobia', 'Unnamed: 4': 'Creencias_religiosas', 'Unnamed: 5': 'Disfobia', 'Unnamed: 6': 'Discriminacion_generacional', 'Unnamed: 7': 'Discriminacion_por_enfermedad', 'Unnamed: 8': 'Discriminacion_sexo_genero', 'Unnamed: 9': 'Ideologia', 'Unnamed: 10': 'Islamofobia', 'Unnamed: 11': 'Orientacion_sexual', 'Unnamed: 12': 'Racismo', 'Unnamed: 13': 'Total_odio'})
delitosodio = delitosodio.iloc[3:14]

In [ ]:
for col in delitosodio.columns:
    try:
        delitosodio[col] = delitosodio[col].astype(float)
        
        if (delitosodio[col] % 1 == 0).all():
            delitosodio[col] = delitosodio[col].astype(int)

    except:
        pass

#### - Limpieza y normalización de Detenciones

In [ ]:
detenciones_periodo = detenciones_periodo.dropna()
detenciones_periodo = detenciones_periodo.rename(columns={'PORTAL ESTADÍSTICO DE CRIMINALIDAD': 'Fecha', 'Unnamed: 1': 'Infracciones_Penales'})

In [ ]:
detenciones_periodo = detenciones_periodo.iloc[1:12]


In [ ]:
for col in detenciones_periodo.columns:
    try:
        detenciones_periodo[col] = detenciones_periodo[col].astype(float)
        
        if (detenciones_periodo[col] % 1 == 0).all():
            detenciones_periodo[col] = detenciones_periodo[col].astype(int)

    except:
        pass

#### - Limpieza y normalización de Victimas por Sexo

In [ ]:
victimas_sexo = victimas_sexo.dropna()
victimas_sexo = victimas_sexo.rename(columns={'PORTAL ESTADÍSTICO DE CRIMINALIDAD': 'Fecha', 'Unnamed: 1': 'Hombres', 'Unnamed: 2': 'Mujeres'})

In [ ]:
victimas_sexo = victimas_sexo.iloc[1:11]

In [ ]:
for col in victimas_sexo.columns:
    try:
        victimas_sexo[col] = victimas_sexo[col].astype(float)
        
        if (victimas_sexo[col] % 1 == 0).all():
            victimas_sexo[col] = victimas_sexo[col].astype(int)

    except:
        pass

In [ ]:
infracciones_SQL = pd.merge(victimas_sexo, detenciones_periodo, on='Fecha', how= 'inner')

### Correlaciones entre las columnas de los Datasets

#### - Correlacion de Delitos de Violencia de Genero

In [ ]:
corr1 = vg.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr1, annot=True, cmap="coolwarm", linewidths=0.5)
plt.show()

#### - Correlacion de Ciberseguridad

In [ ]:
corr2 = ciberseguridad.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr2, annot=True, cmap="coolwarm", linewidths=0.5)
plt.show()

#### - Correlacion de Delitos de Odio

In [ ]:
corr3 = odio_periodo.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr3, annot=True, cmap="coolwarm", linewidths=0.5)
plt.show()

#### - Correlacion de Victimas de Infracciones

In [ ]:
corr4 = infracciones_SQL.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr4, annot=True, cmap="coolwarm", linewidths=0.5)
plt.show()

### Uniones del Dataset mediante merge para tener una tabla final con los totales

#### - Selección de las columnas de cada dataset que vamos a querer para tabla central del SQL y unión de datasets por la columna del 'Fecha'
#### - Conservación de dataset antiguos para poder meterlos como tablas complementarias

In [ ]:
merge_1 = pd.merge(historico, pobreza_historicos, on='Fecha', how= 'inner')

In [ ]:
merge_2 = pd.merge(merge_1, salario_historico, on= 'Fecha', how= 'inner')

In [ ]:
merge_3 = pd.merge(merge_2, allanamientos_periodo, on= 'Fecha', how= 'inner')

In [ ]:
merge_4 = pd.merge(merge_3, ciberseguridad, on= 'Fecha', how= 'inner')

In [ ]:
merge_5 = pd.merge(merge_4, odio_periodo, on= 'Fecha', how= 'inner')

In [ ]:
SQL = pd.merge(merge_5, detenciones_periodo, on= 'Fecha', how= 'inner')

In [ ]:
SQLfinal = SQL[["Fecha", "Poblacion_total", 'SMI', 'Total_cibercriminalidad', 'Total_de_Delitos_de_Odio', 'Infracciones_Penales', ' Allanamientos_totales']]

### Creación de CSVs para poder llevarlo a SQL
#### - Tabla Principal: SQLfinal
#### - Resto de Tablas: 'nombre_de_la_tabla'_SQL

In [ ]:
SQLfinal.to_csv("CSV SQL/SQLfinal.csv", index=False, encoding="utf-8-sig")

In [ ]:
ciberseguridad.to_csv('CSV SQL/ciberseguridad_SQL.csv', index=False, encoding="utf-8-sig")
vg.to_csv('CSV SQL/VG_SQL.csv', index=False, encoding="utf-8-sig")
odio_periodo.to_csv('CSV SQL/Odio_SQL.csv', index=False, encoding="utf-8-sig")
victimas_sexo.to_csv('CSV SQL/Victimas_SQL.csv', index=False, encoding="utf-8-sig")
infracciones_SQL.to_csv('CSV SQL/Infracciones_SQL.csv', index=False, encoding="utf-8-sig")


In [54]:
delitosodio.to_csv('CSV SQL/DelitosOdio_SQL.csv', index=False, encoding="utf-8-sig")